## Polygenic Scores for Insecticide Resistance Surveillance

- [Introductory slides](https://docs.google.com/presentation/d/1qpefQbitedxnI68M82BNK2bZGOC6cPDSXSumE1DpMPY/edit?usp=sharing)
- [Git Repo](https://github.com/LSTM-VIGG/pgs)

In this workshop, we will explore how **polygenic scores (PGS)** can be used to quantify and track **insecticide resistance** in malaria vector populations.    

We will work with MalariaGEN-derived genomic data (`malariagen_data`) and use effect size estimates from a recent targeted amplicon sequencing study using the Ag-vampIR + AmpSeeker platform ([Nagi et al., 2025, *bioRxiv*](https://www.biorxiv.org/content/10.1101/2025.02.14.637727v2)).

This notebook will guide you through:
- Understanding and interpreting polygenic scores in a vector genomics context  
- Using effect size data (odds ratios) to compute polygenic scores  
- Applying PGS to individual and population (cohort) mosquito genotype datasets  

#### Why Use Polygenic Scores for Resistance Surveillance?

Resistance to insecticides is typically polygenic in nature; many loci contribute to the resistant phenotype. However, traditional resistance diagnostics focus on *single* mutations (e.g. Vgsc-995F) which capture only a small fraction of phenotypic variation, and therefore their actual utility for disease control programmes is limited. Advances in whole-genome and targeted sequencing are now making it possible to affordably genotype mosquitoes at many loci across the genome, but to take advantage of this, we need methods that can summarise the phenotypic effect of those genotypes.

#### What Are Polygenic Scores?

A **polygenic score (PGS)** — sometimes called a genetic risk score (GRS) — is a single numerical value summarizing the *cumulative contribution* of multiple genetic variants to a particular phenotype.  

For insecticide resistance, this phenotype reflects the ability of a mosquito to survive exposure to an insecticide. Each single-nucleotide polymorphism (SNP) or mutation has an **effect size (β, log odds ratio)** that indicates its strength and direction of association with resistance to a given insecticide. To calculate the polygenic score, we simply multiply the effect size by the genotype dosage (0,1,2) and sum across variants.

For an individual mosquito $i$, the polygenic score can be represented mathematically as:

$PGS_i = \sum_{j=1}^{m} \beta_j \times G_{ij}$

where:  
- $m$ = number of variants included  
- $beta_j$ = estimated *effect size* (e.g. log odds ratio) of variant $j$
- $G_{ij}$ = genotype dosage (0, 1, or 2 copies of the resistance allele) for individual $i$

This creates a quantitative summary measure that captures small, additive contributions from many loci across the genome — providing a framework for polygenic forms of resistance that cannot be explained by single-point mutations alone.

Polygenic scores provide several key advantages:

- **Interpretable:** Single easy to interpret value to communicate to policy-makers.
- **Standardised:** Enable simple comparisons across studies, populations, and time points.
- **Quantitative:** Translate genetic information into a measurable, continuous trait.

<figure style="text-align: center;">
  <img src="https://github.com/LSTM-VIGG/pgs/blob/main/pgs-fig1.png?raw=1" alt="Figure placeholder for PGS schematic" width="600">
  <figcaption><em>Conceptual overview of polygenic score construction for insecticide resistance.</em></figcaption>
</figure>

---

### Workshop Overview

In this introductory notebook, we will cover:

1. **Data Import** — Loading `malariagen_data` genotypes for target SNPs.  
2. **Effect Sizes** — Importing and inspecting effect sizes (β values / log odds ratios) from the Ag-vampIR + AmpSeeker study.  
3. **PGS Calculation** — Computing per-individual scores from allele dosages and effect sizes.  
4. **Population-Level Summaries** — Aggregating scores to obtain population or cohort-level *mean PGS* values.  
5. **Visualization & Interpretation** — Plotting PGS distributions, assessing variation across geography or time.

In [ ]:
%pip install malariagen_data -qq

In [ ]:
import malariagen_data
import numpy as np
import pandas as pd
import allel
from tqdm.notebook import tqdm
!wget -q https://raw.githubusercontent.com/LSTM-VIGG/pgs/refs/heads/main/pgs_helpers.py
import pgs_helpers as pgs

ag3 = malariagen_data.Ag3()
ag3

#### Load effect size data

Lets load the effect size data. The key columns we will need are the `odds_ratio` column and `snp_id`, which tells us the contig, position and alternate allele of the resistance mutation. We can also see other information, such as the p-value and whether this SNP passed FDR control.

In [ ]:
df_effects = pd.read_csv("https://raw.githubusercontent.com/LSTM-VIGG/pgs/refs/heads/main/effect-sizes-siaya.csv", index_col=0)
df_effects = df_effects.set_index('snp_id')
df_effects.head()

In [ ]:
df_effects.shape

#### Load genotype data for these SNPs with malariagen_data

In [ ]:
# define sample sets and query
sample_set = 'AG1000G-UG'
sample_query = "taxon == 'gambiae'"

# load sample metadata
df_samples = ag3.sample_metadata(sample_set, sample_query)
df_samples.shape

Lets retrieve snp_calls for every SNP in our effect size data. We will use the `snp_id` column to get the contig, position and alternate allele for each SNP, then call malariagen_data.snp_calls. Because we have relatively few SNPs to retrieve (37), I will loop through each SNP and retrieve them one at a time.

In [ ]:
loc

In [ ]:
# setup lists to hold genotype arrays, alt alleles and positions
gns = []
alts = []
poss = []

# loop through each SNP in the effect size data
for i, row in tqdm(df_effects.iterrows()):

    # get the region string from the snp_id
    loc = row.name.replace("snp_", "").split("_")[0]
    region = f"{loc}-{loc.split(":")[1]}"

    # get the snp calls for this region
    ds_snps = ag3.snp_calls(region=region, sample_sets=sample_set, sample_query=sample_query)
    gn = allel.GenotypeArray(ds_snps['call_genotype'].values)
    alt = ds_snps['variant_allele'].values
    pos = ds_snps['variant_position'].values

    gns.append(allel.GenotypeDaskArray(ds_snps["call_genotype"].data))
    alts.append(alt)
    poss.append(pos)

# concatenate the genotype arrays, alt alleles and positions
gn = gns[0].concatenate(gns[1:], axis=0).compute()
alts = np.concatenate(alts).astype(str)
pos = np.concatenate(poss)
samples = ds_snps['sample_id'].values

#### Calculate the number of resistance alleles (genotype dosage) for each variant/individual

As we are working with relatively few SNPs, I'm going to use pandas dataframes to hold the genotype data. For larger numbers of SNPs (for example, from a GWAS), you would want to use a more efficient data structure, such as scikit-allel's `allel.GenotypeArray`.

In [ ]:
# create a dataframe with contig, pos, ref, alt information
contigs = df_effects.index.str.split(":").str.get(0).str.replace("snp_", "").values
df_var = pd.DataFrame({'contig': contigs, 'pos': pos, 'ref': alts[:, 0].astype(str), 'alt': np.apply_along_axis(lambda x: ','.join(x), axis=1, arr=alts[:, 1:].astype(str))})

# create a dataframe with genotypes
df_geno = pd.DataFrame(gn.to_gt().astype(str), columns=samples)

# combine var_df and df_geno
df_geno = pd.concat([df_var, df_geno], axis=1)
df_geno.head(3)

As we have some alleles which are multiallelic, we will need to split these into multiple rows, one for each alternate allele. We will then convert the genotype strings (e.g. "0/1") into a count of the number of alternate alleles (0, 1 or 2).

In [ ]:
df_geno = pgs.split_rows_with_multiple_alleles(df_geno, samples)
df_geno = pgs.convert_genotype_to_alt_allele_count(df_geno, samples)

# create a snp_id column and set as index, and drop unneeded columns
df_geno = df_geno.assign(snp_id=lambda x: "snp_" + x.contig + ":" + x.pos.astype(str) + "_" + x.alt)
df_geno = df_geno.set_index('snp_id')
df_geno = df_geno.drop(columns=['contig', 'pos', 'ref', 'alt'])
df_geno.head(3)

#### Remove SNPs in high LD

In [ ]:
target_snps = df_effects.index.to_list()
df_geno = df_geno.loc[target_snps]

retained_snps = pgs.linked(df_geno.T, df_effects)

In [ ]:
retained_snps

In [ ]:
df_geno.shape

In [ ]:
# df_geno.reset_index().query("snp_id in @selected_snps")

In [ ]:
df_geno_unlinked = df_geno[:, retained_snps.tolist()]
df_geno_unlinked

Now we can select only the snp_ids which are in our effect size data, and calculate PGS.

In [ ]:
df_pgs = pgs.calculate_pgs(df_geno_unlinked, df_effects, df_samples)
df_pgs.head()

Lets look at the distribution of PGS values across individuals.

In [ ]:
df_pgs.pgs.hist()